In [1]:
from sage.algebras.flag_algebras import *
import itertools

In [12]:
#This is code to generate unordered colors for 3-graphs without K4 and monochromatic edge

def _identifyT(n, ftype_points, edges, A, B, C):
    g = Graph([list(range(n+len(edges)+3)), [(i+n,x) for i,b in enumerate(edges) for x in b] + 
               [(a[0], n+len(edges)) for a in A] + 
               [(b[0], n+len(edges)+1) for b in B] + 
               [(c[0], n+len(edges)+2) for c in C]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))] + [[n+len(edges), n+len(edges)+1, n+len(edges)+2]]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateT(n):
    ThreeGraphTheory.exclude(ThreeGraphTheory(4, edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3], [1, 2, 3]]))
    for xx in ThreeGraphTheory.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        for yy in itertools.product(range(3), repeat=int(n-1)):
            good = True
            yy = list(yy) + [0]
            for ee in edges:
                if len(set([yy[ii] for ii in ee]))==1:
                    good = False
                    break
            if good:
                A = [[ii] for ii, oo in enumerate(yy) if oo==0]
                B = [[ii] for ii, oo in enumerate(yy) if oo==1]
                C = [[ii] for ii, oo in enumerate(yy) if oo==2]
                iden = _identifyT(n, [], edges, A, B, C)
                if iden not in unique:
                    unique.append(iden)
                    yield {'edges': edges, 'A': A, 'B': B, 'C':C}

CTGT = CombinatorialTheory('UnOrd3GNoMonoNoK4', _generateT, _identifyT, edges=3, A=1, B=1, C=1)

In [ ]:
TGT = ThreeGraphTheory
#CTGT as defined above

#Set the target size, for this short test it will be 5
target_size = 5

#The degree difference and min degree is a combination of multiple flags
#listing each coloring would be really tedious, so I wrote some script
#that replaces ThreeGraph flags with all possible ColoredThreeGraph flags

#This creates an uncolored flag from a colored one
def forget_colors(uncolor_theory, flag):
    return uncolor_theory(flag.size(), ftype=flag.ftype_points(), edges=flag.blocks()["edges"])

#This creates a colored flag vector from uncolored, such that after forgetting the color they are equal
def colored(color_theory, uncolor_theory, flagvecs):
    if not isinstance(flagvecs, list):
        flagvecs = [flagvecs]
    flagvecs = [xx.afae() for xx in flagvecs]
    if len(flagvecs)==0:
        return []
    n = flagvecs[0].size()
    colored_flag_list = color_theory.generate_flags(n)
    res = [0]*len(flagvecs)
    for colored_flag in colored_flag_list:
        uncolored_flag = forget_colors(uncolor_theory, colored_flag)
        for ii in range(len(flagvecs)):
            res[ii] += flagvecs[ii][uncolored_flag]*colored_flag
    return res

#This constructs the degree difference on uncolored graphs. This will be an equality
nc10 = TGT(4, ftype=[0, 1], edges=[[0, 2, 3]])
nc11 = TGT(4, ftype=[0, 1], edges=[[1, 2, 3]])
nc20 = TGT(4, ftype=[0, 1], edges=[[0, 1, 2], [0, 2, 3]])
nc21 = TGT(4, ftype=[0, 1], edges=[[0, 1, 2], [1, 2, 3]])
nc30 = TGT(4, ftype=[0, 1], edges=[[0, 1, 2], [0, 1, 3], [0, 2, 3]])
nc31 = TGT(4, ftype=[0, 1], edges=[[0, 1, 2], [0, 1, 3], [1, 2, 3]])
degree_difference = nc10 - nc11 + nc20 - nc21 + nc30 - nc31

#This is the min degree condition also on uncolored graphs. This will be just an inequality
min_degree = TGT(3, ftype=[0], edges=[[0, 1, 2]]) - 5/9

#Create a list from the uncolored positive constraints
uncolored_positives = [degree_difference, -degree_difference, min_degree]


#This constraint is for balanced coloring. This is used as an equality
same_color = CTGT(2, ftype=[0], A=[[0], [1]]) - 1/3
#Since it is already colored, add it to the positives list
positives = [same_color, -same_color]

#Construct the positive constraints form the uncolored ones
#First make them have the correct size and project them to empty ftype
projected_uncolored_positives = []
for fv in uncolored_positives:
    d = target_size - fv.size()
    k = fv.ftype().size()
    terms = fv.afae().parent().generate_flags(k+d)
    projected_uncolored_positives += [fv.mul_project(xx) for xx in terms]
positives += colored(CTGT, TGT, projected_uncolored_positives)

#While it is not too hard to write the edge with colored variants, the colored code works here too
edge = TGT(3, edges=[[0, 1, 2]])
colored_edge = colored(CTGT, TGT, edge)[0]

CTGT.optimize_problem(colored_edge, target_size, maximize=True, positives=positives)

In [ ]:
#For 3 colored 3-graphs without mono edge
#This is the unordered, not that useful as the number of flags is huge
#Just some leftover code, don't run it

def _identifyT(n, ftype_points, edges, A, B):
    g = Graph([list(range(n+len(edges)+2)), [(i+n,x) for i,b in enumerate(edges) for x in b] + 
               [(a[0], n+len(edges)) for a in A] + 
               [(b[0], n+len(edges)+1) for b in B]], 
              format='vertices_and_edges')
    partt = [[ii] for ii in ftype_points] + \
            [[ii for ii in range(n) if ii not in ftype_points]] + \
            [list(range(n,n+len(edges)))] + [[n+len(edges)], [n+len(edges)+1]]
    blocks = tuple(g.canonical_label(partition=partt).edges(labels=None, sort=True))
    ftype_points = tuple(range(len(ftype_points)))
    return (n, ftype_points, blocks)

def _generateT(n):
    ThreeGraphTheory.exclude(ThreeGraphTheory(4))
    for xx in ThreeGraphTheory.generate_flags(n):
        unique = []
        edges = xx.blocks()['edges']
        for yy in itertools.product(range(3), repeat=int(n)):
            good = True
            for ee in itertools.combinations(range(n), 3):
                if list(ee) in edges:
                    continue
                elif len(set([yy[ii] for ii in ee]))==1:
                    good = False
                    break
            if good:
                A = [[ii] for ii, oo in enumerate(yy) if oo==1]
                B = [[ii] for ii, oo in enumerate(yy) if oo==2]
                iden = _identifyT(n, [], edges, A, B)
                if iden not in unique:
                    unique.append(iden)
                    yield {'edges': edges, 'A': A, 'B': B}

T = CombinatorialTheory('Ord3GNoMonoNoK4', _generateT, _identifyT, edges=3, A=1, B=1)

#should be 240409 (too large)
#len(T.generate_flags(6))